# Experiment Notebook: Sanity Checks

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: System Dynamics Over Volatile Asset Price Trajectories](#Analysis-1:-System-Dynamics-Over-Volatile-Asset-Price-Trajectories)
* [Analysis 2.1: State Variable Relationships](#Analysis-2.1:-State-Variable-Relationships)
* [Analysis 2.2: State Variable Correlation](#Analysis-2.2:-State-Variable-Correlation)
* [Analysis 3: Liquidity Pool Dynamics](#Analysis-3:-Liquidity-Pool-Dynamics)

# Experiment Summary 

The purpose of this notebook is to perform a set of sanity checks that validate the expected key system dynamics, as well as the relationships between different key state variables. These analyses should also serve as an educational explanatory tool for less intuitive system dynamics and as an introduction to the model.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
from experiments.notebooks.helpers.system_metrics import *

In [ ]:
# Configure Plotly
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False
logging.disable(logging.DEBUG)

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
# simulation_1.model.initial_state.update({})
# simulation_1.model.params.update({})

# Analysis 1: System Dynamics Over Volatile Asset Price Trajectories

The Fei Protocol Model is strongly driven by the Volatile Asset price process - mimicking the dependency on the Ethereum price which avenues of FEI and PCV have in the system.

In this analysis we look at the effect of setting the Volatile Asset price trajectory in four scenarios:
- As a linear constant price
- As a step price
- As a linearly increasing price
- As a linearly decreasing price

The purpose of this analysis is to illustrate and validate the driving influence of the Volatile Asset price on key system dynamics.

In [ ]:
# Analysis-specific setup
simulation_1.model.params.update({
    # Disable policy by setting to `None`
    "target_stable_pcv_ratio": [None],
    "target_stable_backing_ratio": [None],
    "volatile_asset_price_process": [
        lambda _run, _timestep: 2_000,
        lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365),
    ],
})

scenario_names = {0: "Constant Price", 1: "Step Price", 2: "Increasing Price", 3: "Decreasing Price"}

In [ ]:
# Experiment execution
df_1, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

In [ ]:
df_1["Scenario"] = df_1["subset"].map(scenario_names)

In [ ]:
fig = df_1.plot(y='volatile_asset_price', color='Scenario')

fig.update_layout(
    title="Volatile Asset (e.g. ETH) Price Trajectories",
    xaxis_title="Timestamp",
    yaxis_title="Volatile Asset Price (USD)",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
from experiments.notebooks.visualizations.plotly_theme import cadlabs_colorway_sequence
color_cycle = itertools.cycle(cadlabs_colorway_sequence)


column_titles = list(scenario_names.values())
row_titles = [
    "Volatile Asset Price (USD)",
    "Stable / Volatile PCV (USD)",
    "Total FEI Supply (FEI)",
    "Collateralization (%)",
    "Liquidity Pool TVL (USD)",
]

fig = make_subplots(
    rows=5,
    cols=len(df_1.subset.unique()),
    shared_xaxes=True,
    shared_yaxes="rows",
    horizontal_spacing=0.05,
    vertical_spacing=0.05,
    column_titles=column_titles,
    row_titles=row_titles,
)

for subset in df_1.subset.unique():
    df_plot = df_1.query('subset == @subset')
    
    show_legend = bool(subset == 0)
    column = subset + 1
    
    row = 1
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.volatile_asset_price,
            name="Volatile Asset Price",
            line=dict(color=cadlabs_colorway_sequence[row - 1]),
            showlegend=show_legend,
        ),
        row=row, col=column,
    )
    
    row = 2
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_volatile_asset_pcv,
            name="Volatile Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[row - 1]),
            showlegend=show_legend,
            stackgroup='one',
        ),
        row=row, col=column
    )
    
    row = 2
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_stable_asset_pcv,
            name="Stable Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[row]),
            showlegend=show_legend,
            stackgroup='one',
        ),
        row=row, col=column
    )
    
    row = 3
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_fei_supply,
            name="Total FEI Supply",
            line=dict(color=cadlabs_colorway_sequence[row]),
            showlegend=show_legend,
        ),
        row=row, col=column
    )
    
    row = 4
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.collateralization_ratio,
            name="Collateralization Ratio",
            line=dict(color=cadlabs_colorway_sequence[row]),
            showlegend=show_legend,
        ),
        row=row, col=column
    )
    
    row = 5
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.liquidity_pool_tvl,
            name="Liquidity Pool TVL",
            line=dict(color=cadlabs_colorway_sequence[row]),
            showlegend=show_legend,
        ),
        row=row, col=column
    )

fig.update_layout(title="Fei Protocol Model Sanity Checks")
fig.show(height=1200)

By looking at the components of total FEI supply in the subset with a step decrease in the volatile asset price, we can further develop our intuition about the behaviour of the model:

In [ ]:
df_debug = df_1.query("subset == 1")

In [ ]:
df_debug["fei_money_market_pcv_deposit_balance_idle"] = df_debug["fei_money_market_pcv_deposit_balance"] * (1 - df_debug["fei_money_market_utilization"])
df_debug["fei_money_market_user_deposit_balance_idle"] = df_debug["fei_money_market_user_deposit_balance"] * (1 - df_debug["fei_money_market_utilization"])

fei_state_variables = [
    # Protocol
    "fei_idle_pcv_deposit_balance",
    "fei_liquidity_pool_pcv_deposit_balance",
    "fei_money_market_pcv_deposit_balance_idle",
    # User
    "fei_idle_user_deposit_balance",
    "fei_savings_user_deposit_balance",
    "fei_liquidity_pool_user_deposit_balance",
    "fei_money_market_user_deposit_balance_idle",
    "fei_money_market_borrowed",   
]

fig = px.area(df_debug, y=fei_state_variables, title="Total FEI Supply")

fig.update_layout(
    title="Total FEI Supply",
    xaxis_title="Timestamp",
    yaxis_title="Total FEI Supply (FEI)",
    legend=dict(title=""),
)
fig.show()

## Effect of Volatile Asset Price Trajectory on Main KPIs

In [ ]:
fn_dict = get_fn_dict()

### 1. Collateralization Ratio

In [ ]:
fig = df_1.plot(
    y='collateralization_ratio_pct',
    color='Scenario'
)

fig.update_layout(
    title="Collateralization Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio (%)",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
compute_metric_set_for_variable(df_1, fn_dict, 'collateralization_ratio')

### 2. Stable Backing Ratio

In [ ]:
fig = df_1.plot(
    y='stable_backing_ratio_pct',
    color='Scenario'
)

fig.update_layout(
    title="Stable Backing Ratio",
    xaxis_title="Timestamp",
    yaxis_title="Stable Backing Ratio (%)",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
compute_metric_set_for_variable(df_1, fn_dict, 'stable_backing_ratio')

### 3. Total PCV

In [ ]:
fig = df_1.plot(
    y='total_pcv',
    color='Scenario'
)

fig.update_layout(
    title="Total PCV",
    xaxis_title="Timestamp",
    yaxis_title="Total PCV (USD)",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
compute_metric_set_for_variable(df_1, fn_dict, 'total_pcv')

### 4. Total User-circulating FEI

In [ ]:
fig = df_1.plot(
    y='total_user_circulating_fei',
    color='Scenario'
)

fig.update_layout(
    title="Total User-circulating FEI",
    xaxis_title="Timestamp",
    yaxis_title="Total User-circulating FEI (FEI)",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
compute_metric_set_for_variable(df_1, fn_dict, 'total_user_circulating_fei')

# Analysis 2.1: State Variable Relationships

The purpose of this analysis is to illustrate and validate the relationships between and distribution of key system state variables using the model's default experiment initial state and system parameters.

In [ ]:
# Experiment execution
df_2, exceptions = run(simulation_2)

In [ ]:
# Post-processing and visualizations

In [ ]:
variables = [
    "volatile_asset_price",
    "total_pcv",
    "collateralization_ratio_pct",
    "total_user_circulating_fei",
]

In [ ]:
fig = px.scatter_matrix(
    df_2,
    dimensions=variables,
    labels={col:col.replace('_', ' ') for col in df_2.columns}, # remove underscore
    opacity=0.25,
)

fig.update_layout(title="Scatter Matrix")
fig.update_traces(diagonal_visible=False)
fig.show(height=1000)

In [ ]:
from model.system_parameters import parameters

capital_allocation_fei_deposit_variables = parameters['capital_allocation_fei_deposit_variables'][0]

capital_allocation_weights = [f"{key}_weight" for key in capital_allocation_fei_deposit_variables]
capital_allocation_balances = [f"{key}_balance" for key in capital_allocation_fei_deposit_variables]

In [ ]:
fig = df_2.groupby('timestep').mean().plot(y=capital_allocation_weights)

fig.update_layout(
    title="FEI Capital Allocation Weights",
    xaxis_title="Timestamp",
    yaxis_title="Weight (Yield / Risk)",
    legend=dict(
        title="",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
fig = px.area(df_2, y=capital_allocation_balances, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation (%)",
    legend=dict(
        title="",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

# Analysis 2.2: State Variable Correlation

The purpose of this analysis is to illustrate and validate the correlation between key system state variables.

In [ ]:
z = df_2[variables + ['timestep']].groupby('timestep').mean().corr()

correlation_matrix = ff.create_annotated_heatmap(
    z=np.round(z, 2).to_numpy(),
    x=z.columns.tolist(),
    y=z.index.tolist(),
    colorscale='Viridis',
    zmax=1, zmin=-1,
    showscale=True,
)

correlation_matrix.update_layout(
    title="Correlation Matrix",
    title_x=0.5,
    autosize=False,
    width=1000,
    height=1000,
)
correlation_matrix.show()

# Analysis 3: Liquidity Pool Dynamics

The purpose of this analysis is to illustrate and validate the effect of a step change in the Volatile Asset price on the liquidity pool imbalance, resulting minting and redemption, and capital allocation.

Evaluating the second subset, a step in the volatile asset price, we expect the following results:

1. A step in the volatile asset price results in an imbalance in the liquidity pool, which is rebalanced by:
 - Step down in volatile asset price results in an excess of FEI in the pool: FEI is released from the liquidity pool into the circulating supply and **redeemed**
 - Step up in volatile asset price results in a deficit of FEI in the pool: FEI is **minted** and deposited into the liquidity pool
2. The rebalancing transaction volume causes a spike in transaction fees collected and resulting liquidity pool yield rate
3. The capital allocation weights for the liquidity pool increase
4. The capital allocation Model moves user FEI into the liquidity pool

In [ ]:
df_subset_1 = df_1.query("subset == 1")

In [ ]:
fig = df_subset_1.plot(y=['fei_minted_redeemed'])

fig.update_layout(
    title="Liquidity Pool FEI Minted (+ve) / Redeemed (-ve)",
    xaxis_title="Timestamp",
    yaxis_title="FEI Minted / Redeemed (FEI)",
    showlegend=False,
)
fig.show()

In [ ]:
fig = df_subset_1.plot(y=['fei_liquidity_pool_pcv_deposit_balance', 'fei_liquidity_pool_user_deposit_balance'])

fig.update_layout(
    title="Liquidity Pool FEI Balance",
    xaxis_title="Timestamp",
    yaxis_title="Balance (FEI)",
    legend=dict(
        title="",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
fig = df_subset_1.plot(y=['volatile_liquidity_pool_pcv_deposit_balance', 'volatile_liquidity_pool_user_deposit_balance'])

fig.update_layout(
    title="Liquidity Pool Volatile Asset Balance",
    xaxis_title="Timestamp",
    yaxis_title="Balance (Volatile Asset Units)",
    legend=dict(
        title="",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
fig = df_subset_1.plot(
    title='Liquidity Pool Invariant',
    x='timestep',
    y=['liquidity_pool_invariant'],
)

fig.update_layout(
    title="Liquidity Pool Invariant",
    xaxis_title="Timestamp",
    yaxis_title="Invariant (FEI * Volatile Asset Balance)",
    showlegend=False,
)
fig.show()

In [ ]:
fig = df_subset_1.groupby('timestep').mean().plot(y=capital_allocation_weights)

fig.update_layout(
    title="FEI Capital Allocation Weights",
    xaxis_title="Timestamp",
    yaxis_title="Weight (Yield / Risk)",
    legend=dict(
        title="",
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()